# Single-Shot Readout Calibration for UA Q2

### Using Experiment and Analysis Class, make sure to update qubit #

In [1]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig, LoopbackInterface, generate_qua_script
from qm.octave import *
from configuration import *
from scipy import signal
from qualang_tools.bakery import baking
from qm.octave import QmOctaveConfig
from quam import QuAM
from scipy.io import savemat, loadmat
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
#from qutip import *
from typing import Union
from macros import *
import datetime 
import os
import time
import warnings
import json
import matplotlib.pyplot as plt
import numpy as np
import Labber
import xarray as xr

from ExperimentClass import ExperimentHandle
from AnalysisClass import AnalysisHandle

warnings.filterwarnings("ignore")

qubit_index = 0 # for UA Q2
TLS_index = 0 # for TLS of interest
dc_flux_index = 1 # for dc line

2024-05-14 12:20:21,332 - qm - INFO     - Starting session: 11bb7543-8e4d-4d04-854c-e7bc4ef9b90e


In [2]:
Experiment = ExperimentHandle()
Analysis = AnalysisHandle("quam_state_q1.json")
machine = Analysis.get_machine()

In [ ]:
# initialize Labber-controlled hardware
# get values from QDAC first
machine = Experiment.set_Labber.get_value_from_QDAC(machine)
# set all Labber settings
machine = Experiment.set_Labber.set_Labber(machine, qubit_index)
# set octave to external clock
Experiment.set_octave.set_clock(machine)

In [ ]:
machine.octaves[0].LO_sources[1].LO_frequency = machine.qubits[qubit_index].f_01 + 50e6
machine = Analysis.set_machine(machine)

In [ ]:
# octave calibration
machine = Experiment.set_octave.calibration(machine, qubit_index)

## 1D resonator spectroscopy

In [ ]:
%matplotlib qt
res_freq_sweep = machine.resonators[qubit_index].f_readout + np.arange(-5E6,5E6 + 1.0,0.05E6)
machine, expt_dataset = Experiment.exp1D.RR.rr_freq_ge(machine, res_freq_sweep, qubit_index, n_avg = 1E3, cd_time = 10E3,
                                                    to_simulate = False, simulation_len = 1000, live_plot = True)

In [ ]:
%matplotlib inline
res_freq_phase = Analysis.exp1D.rr_freq_ge(expt_dataset)

In [ ]:
# doesn't have to update. Use res_freq_phase
machine.resonators[qubit_index].f_readout = res_freq + 0E6
machine.octaves[0].LO_sources[0].LO_frequency = machine.resonators[qubit_index].f_readout + 50E6
machine = Experiment.set_octave.calibration(machine, qubit_index)
machine = Analysis.set_machine(machine)

## 1D qubit spectroscopy

In [ ]:
qubit_freq_sweep = machine.qubits[qubit_index].f_01 + np.arange(-30E6, 30E6 + 1, 0.5E6)        
%matplotlib qt
machine, expt_dataset = Experiment.exp1D.Rabi.qubit_freq(machine, qubit_freq_sweep, qubit_index, pi_amp_rel = 1.0, to_plot = False,
                                                         ff_amp = 0.0, n_avg = 1E3, cd_time = 30E3, simulate_flag = False, simulation_len = 1000)

In [ ]:
%matplotlib inline
qubit_freq = Analysis.exp1D.peak_fit(expt_dataset,method="Gaussian")
machine.qubits[qubit_index].f_01 = qubit_freq + 0E6
machine.octaves[0].LO_sources[1].LO_frequency = machine.qubits[qubit_index].f_01 + 50e6
machine = Experiment.set_octave.calibration(machine, qubit_index)
machine = Analysis.set_machine(machine)

## Measure single-shot I, Q blobs

In [ ]:
%matplotlib inline
machine, expt_dataset = Experiment.exp1D.RR.single_shot_IQ_blob(machine, qubit_index, n_avg = 30E3, cd_time = 30E3, live_plot = False)

In [ ]:
%matplotlib inline
angle, threshold, fidelity, gg, ge, eg, ee = Analysis.exp1D.two_state_discriminator(expt_dataset)

In [ ]:
# set the rotation angle
machine.resonators[qubit_index].rotation_angle -= angle # note this is the change value not the absolute value!
machine.resonators[qubit_index].ge_threshold = threshold
machine = Analysis.set_machine(machine)

## optimize readout duration and amp

In [ ]:
%matplotlib qt
res_duration_sweep_abs = np.arange(300, 2000 + 1, 20)
res_amp_sweep = np.arange(0.05,0.5,0.002)
machine, res_amp_sweep, res_duration_sweep_abs, sig_amp = Experiment.exp2D.RR.rr_pulse_optimize(res_duration_sweep_abs, res_amp_sweep, qubit_index, res_index, flux_index, 
                    n_avg=10E3, cd_time=20E3, simulate_flag=False, simulation_len=1000, plot_flag=True, machine=machine)

In [ ]:
machine.resonators[res_index].readout_pulse_amp = 0.5
machine.resonators[res_index].readout_pulse_length = 1096
machine = Analysis.set_machine(machine)

## optimize single-shot frequency

In [ ]:
res_freq_sweep = machine.resonators[qubit_index].f_readout + np.arange(-1.5E6, 1.5E6 + 1, 0.02E6)

%matplotlib qt
machine, expt_dataset = Experiment.exp1D.RR.single_shot_freq_optimization(machine, res_freq_sweep, qubit_index, n_avg = 2E3, cd_time = 25E3, 
                                                           final_plot = True, live_plot = True)

In [ ]:
%matplotlib inline
res_freq = Analysis.exp1D.peak_fit(expt_dataset,method="Gaussian", SNR = True)

In [ ]:
machine.resonators[qubit_index].f_readout = res_freq + 0E6
machine.octaves[0].LO_sources[0].LO_frequency = machine.resonators[qubit_index].f_readout + 50E6
machine = Analysis.set_machine(machine)
# octave calibration
machine = Experiment.set_octave.calibration(machine, qubit_index)

# These are from QM, and I don't think they work very well...

## optimize single-shot amplitude

In [ ]:
res_amp_sweep = np.arange(0.5,1.5 + 1E-4, 0.01)

%matplotlib qt
machine, expt_dataset = Experiment.exp1D.RR.single_shot_amp_optimization(machine, res_amp_sweep, qubit_index,
                              n_avg = 5E3, cd_time = 25E3, final_plot= True)

In [ ]:
%matplotlib inline
readout_amp = Analysis.exp1D.peak_fit_fidelity(expt_dataset,method="Gaussian")

In [ ]:
machine.resonators[qubit_index].readout_pulse_amp = 0.45
machine = Analysis.set_machine(machine)

In [ ]:
Ig = expt_dataset.Ig.values

In [ ]:
Ig.shape

## optimize single-shot readout duration

In [3]:
division_length = 40
%matplotlib qt
machine, expt_dataset = Experiment.exp1D.RR.single_shot_duration_optimization(machine, division_length, qubit_index,
                                                                    readout_len = 2000, n_avg = 10E3, cd_time = 25E3)

Integration weights chunk-size length in ns: 40
The readout has been sliced in the following number of divisions 50
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.91s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.92s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.94s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.95s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.97s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.98s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed time: 0.99s
Progress: [##################################################] 100.0% (n=10000/10000.0) --> elapsed 

In [ ]:
machine.resonators[res_index].readout_pulse_length = 1000
machine = Analysis.set_machine(machine)

In [4]:
expt_dataset

<xarray.Dataset>
Dimensions:         (n: 10000, x: 50)
Coordinates:
    Readout Length  (x) int32 40 80 120 160 200 240 ... 1840 1880 1920 1960 2000
Dimensions without coordinates: n, x
Data variables:
    Ig              (n, x) float64 -0.0004958 -0.0007791 ... -0.01515 -0.016
    Qg              (n, x) float64 -0.0009003 -0.0006356 ... -0.02559 -0.02537
    Ie              (n, x) float64 0.0003867 -4.983e-05 ... 0.01037 0.009754
    Qe              (n, x) float64 -0.001108 -0.001446 ... -0.009625 -0.007582
    Fidelity        (x) float64 56.27 57.94 58.97 60.09 ... 73.01 72.92 72.93
Attributes:
    created:      2024-05-14 12:20:27
    finished:     2024-05-14 12:20:30
    name:         single_shot_len
    long_name:    Readout Length Optimization
    qubit:        ['q1']
    TLS:          []
    sequence:     
    description:  q1_single_shot_len
    directory:    Z:\QM_Data_DF5\2024\05\Data_0514\2024-05-14_q1_single_shot_...